In [1]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [2]:
# task1
import os
import json
from langchain_community.chat_models import AzureChatOpenAI
from pprint import pprint

# Set environment variables
os.environ["OPENAI_API_KEY"] = "6ffhCm6wvgxD2LRD7wNZI5sHgqHn4lqYOY7xn8Ycdg7vHvZ8qyujJQQJ99BCACYeBjFXJ3w3AAABACOGwuFA"
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"

# Initialize Azure OpenAI GPT model
def initialize_chat_model(temperature=0.0):
    return AzureChatOpenAI(
        deployment_name="gpt-4o",
        azure_endpoint="https://tcoeaiteamgpt4o.openai.azure.com/",
        temperature=temperature
    )

# First prompt to process the input data
def process_input_data(data):
    prompt = f"""

      # Input  :
          Read the entire file {data}, it contains thew following : [Question-Answer pairs, Domain, Region, Use case, Article laws, Schema with possible_metrics] listed information, which will be made use for further evaluations.
          #--------------------------------------------------------------------------------------------------------

      # Instructions for interpreting the input file data :
          1. Question : The Responsible AI chat application will post the question to the user, to understand about their system.
          2. Answer : The user provides an answer, describing the details of their application.
          3. Domain : It tells the domain of the application under test that the user refers to.
          4. Region : Region/Countries where the system is existent or deployed.
          5. UseCase : Application functionality is described.
          6. Schema : Holds all possible metrics extracted from Knowledge Graph.
              [possible_metrics : All the relatable/relevant metrics to the answer will be present collectively.]

      #--------------------------------------------------------------------------------------------------------

      # System Definitions :
      As an expert in Generative AI and Responsible AI Assurance, analyze the question, answer, domain, region, and use case.
      Focus primarily on the correlation between these elements to select the most relevant metrics and recommend it for the business analyst to evaluate.
      Article laws should be considered as a secondary check, not a primary driver of metric decisions. They serve as an additional layer of validation
      but should not drastically impact metric selection.
      Also, recommend any additional applicable metrics beyond those listed in the schema.

      #--------------------------------------------------------------------------------------------------------

      # Steps to Identify the Right Metrics :
          1. Correlate the provided details with the metrics and recommend the most relevant ones. Identify the metrics based on the [answer] in the input, while considering the [domain], [region], [usecase], and [possible_metrics] from the [schema].
          2. Think and reason out why this metric will be applicable to this question-answer pair. Provide by a clear justification relevant to the specific question-answer pair.
          3. If you find that the metrics do not align with the provided question-answer pair, feel free to propose a new metric. Ensure that you carefully consider the reasoning and validation steps before suggesting it, and only recommend it if it is relevant to the specific question-answer pair context.
          4.After generating the full list of metrics, revisit and refine the metric names, descriptions. Ensure they include realistic examples and incorporate terms or keywords from the question-answer pair.
          understand the answer provided and add an in-built advice accordingly in the Reason(what to do and what not to do) for better functionality.
          5. The metrics should be semantically aligned with the specific question-answer pair, avoiding any generic reasoning or validation steps.
          6.The metrics description should be relevant to the answer, but it’s important not to be overly specific to the example or scenario given in the response.
          7.Ensure that Provide at least 20 metrics for each question.
      #--------------------------------------------------------------------------------------------------------

    Justification Reason for each metrics : These justifications should explain why the system utilizes each metric to ensure smooth operation, particularly during the increased volume of bookings.
                  - Task Completion Rate (TCR): This metric is chosen to measure how effectively the system completes user requests, ensuring that the increase in bookings during peak months (November and December) results in successful task execution.
                  - Data Completeness Ratio (DCR): This metric ensures that all required booking information (such as user details, dates, destinations, etc.) is collected and used appropriately, especially during high-volume periods when missing data could lead to errors or delays.
                  - Correctness/Factuality: This metric is selected to verify the accuracy of booking details, ensuring that the system delivers precise and error-free booking confirmations, which is crucial during peak booking times for the USA and Canada.



      # Examples to be captured :

          Example 1 :
              Question => How does the system handle the increase in bookings?
              Answer => Our system has more booking on the month of november and december to USA and Canada.
              Metrics =>
              [
                {{
                  "name": "Task Completion Rate (TCR)",
                  "description": "Percentage of users successfully completing tasks with the chatbot, with a notable increase in task completions for bookings",
                  "justification_reason": "Tell why and how this metric is captured based on the inputs (answer)",

                  }},
                {{
                  "name": "Data Completeness Ratio (DCR)",
                  "description": "Ensures all essential details are present in user profiles and booking records.",
                  "justification_reason": "Reason for the second metric."
                    }},
                {{
                  "name": "Correctness/Factuality Rate",
                  "description": "Ensures booking details (e.g., dates, flight numbers) are accurate and match real data",
                  "justification_reason": "Reason for the metric."
                  }}
              ]
          # Output Generation :
              1. Write the Description and Reason in output in such a way that, a wide range of people from different age group,including younger(10 years to 15 years old) should be able to understand and it should make sense.
              2. Do not write any other information in the output other than what is expected.Should not be too complicated and confusing.
              3. Ensure, Reason should not explicitly mention its reading from "the answer", the answer basically tell about the system/application that is being analysed.
              6. Ensure that all the metrics are present in the output.
              7. Make sure the output looks through all the possible_metrics and  ensure to add all of them to the output.
              8. Make sure the output for All the questions and answers should be present in output from the {data}.
              9.Make sure metric include  **name**, **description**, **justification_reason** in the output.



    """

    chat_model = initialize_chat_model()
    response = chat_model.call_as_llm(prompt)
    return response



In [5]:
# Second prompt to process the output of the first response and get final results
def get_final_advisory(data_from_first_response,data):
  # output of first + data
    prompt = f"""
        # Input  :
                        Read the entire {data_from_first_response} and {data}, it contains thew following : [Question-Answer pairs, Domain, Region, Use case, Article laws] listed information, which will be made use for further evaluations.
                        #--------------------------------------------------------------------------------------------------------

                    # Instructions for interpreting the input file data :
                        1. Question : The Responsible AI chat application will post the question to the user, to understand about their system.
                        2. Answer : The user provides an answer, describing the details of their application.
                        3. Domain : It tells the domain of the application under test that the user refers to.
                        4. Region : Region/Countries where the system is existent or deployed.
                        5. UseCase : Application functionality is described.


                          # Steps to Identify the Right Metrics :
                          1. Based on the justifying reason provided, outline the validation steps for testing the system with the metric you suggested. Ensure that you suggest only the most applicable metric, so the reason and validation steps you provide must be accurate.
                          2. Suggest a threshold for each metric within the range of 0 to 1. Provide a reason for why this threshold is appropriate based on the inputs and the specific context of the question-answer pair. Consider the implications of the metric values, where 0 might indicate poor performance and 1 indicates higher performance, and vice versa based on the metric type (e.g., factuality vs. stereotype).
                          3. Explain the impact on the system if the metrics fall below or exceed the suggested threshold, providing a thorough understanding of the use case.
                          4.** Add a **`category`** field for each metric:
                          - **Model Quality:** Metrics that evaluate the quality, performance, or functionality of the application under test.
                          - **Trustworthy:** Metrics that assess the ethical behavior, fairness, transparency, and compliance with laws or guidelines.
                          5. **Category Reason (`category_reason`):** Provide the reason why the metric was tagged as either Model Quality or Trustworthy.
                          6. ** Add an **`applicability`** field for each metric:
                          - **High:** This metric is highly critical, and it is mandatory for the user, tester, or business analyst to monitor it without fail, as skipping it could significantly impact performance, user experience, or business decisions.
                          - **Medium:** This metric is important but not critical. It provides valuable insights that can enhance decision-making, but its absence won’t significantly impact the core functionality or business outcomes.
                          - **Low:** This metric is of minimal importance and is primarily for informational or supplementary purposes. It has little to no direct impact on critical functions or business decisions.
                          7. **Applicability Reason (`applicability_reason`):** Provide the reason why the metric has been assigned the specified applicability level, based on its importance in the context of the system and its potential impact on performance, reliability, or compliance.
                          Reason for the applicability level,must be inclined towards usecase.
                          8. **Make sure the output looks through all the possible_metrics and  ensure to add all of them to the output.**
                          9. **Make sure the output for all the questions and answers should be present as from the {data}.**
                          10.STRICTLY include ALL the questions and answers from the input data {data_from_first_response}.
                          11.For example {data_from_first_response} contains 10 questions then output should STRICTLY contain all those 10 questions.




                   Validation Steps: The validation process ensures that the system operates as expected based on the information provided in the answer.

                            Example Scenario for a business analyst to test in the system:
                                1. For instance, if the app is designed to book flight tickets, a user may request to book a flight from India to Malaysia on April 1 at 9:30 PM IST for two people in economy class.
                                2. The app should respond by confirming the successful booking of the flight, stating the correct details: "Flight from India to Malaysia on April 1 at 9:30 PM IST for two people in economy class has been successfully booked."

                                Validation Steps:
                                 - Task Completion Rate: The system should verify if the task requested by the user (booking the flight) was successfully completed.
                                 - Data Completeness Ratio: The system should ensure that all the necessary data (such as destination, date, number of passengers, class) is correctly collected from the user.
                                 - Correctness Rate: The system should cross-check the user input and the output to confirm that the booking details are correct and match what was requested.


                    #--------------------------------------------------------------------------------------------------------

                    # Output Generation :
                        1. Write the Description and Reason in output in such a way that, a wide range of people from different age group, including younger(10 years to 15 years old) should be able to understand and it should make sense.
                        2. Do not write any other information in the output other than what is expected.
                        Should not be too complicated and confusing.
                        3. Ensure, Reason should not explicitly mention its reading from "the answer", the answer basically tell about the system/application that is being analysed.
                        4. Do not make any changes in article_laws in output, it should remain exactly same as given in input.
    					          5.Ensure that no other category apart from Model Quality" or "Trustworthy is present.
                        6. Ensure that all the metrics are present in the output.
                        7. Make sure the output looks through all the possible_metrics and  ensure to add all of them to the output.
                        8. Make sure the output for all the questions and answers should be present as from the {data_from_first_response}.


                    #--------------------------------------------------------------------------------------------------------

                    # Output Metrices Format [JSON] :

                         [
                          {{
                            "question_no": "question number from the input",
                            "question": "The question mentioned",
                            "answer": "The answer mentioned",
                            "article_laws": [
                              {{
                                "law": "Article name",
                                "article": "Article number",
                                "title": "Title of the Article",
                                "description": "Article description",
                                "reason": "Reason why the article is being identified and mapped for the answers",
                                 "applicability": "High/Medium/Low",
                                "applicability_reason": "Reason for the applicability level
                              }},
                              {{
                                "law": "Article name",
                                "article": "Article number",
                                "title": "Title of the Article",
                                "description": "Article description",
                                "reason": "Reason why the article is being identified and mapped for the answers",
                                 "applicability": "High/Medium/Low",
                                "applicability_reason": "Reason for the applicability level
                              }}
                            ],
                            "metrices": [
                              {{
                                "metric_name": "The name of the metric",
                                "metric_description": "The description of the metric based on the answer",
                                "validation_steps": "Validation steps for this metric",
                                "justification_reason": "Tell why and how this metric is captured based on the inputs (answer)",
    							 "category": "Model Quality" or "Trustworthy",
                            "category_reason": "Explanation of why this metric belongs to the selected category",
                            "applicability": "High/Medium/Low",
                                "applicability_reason": "Reason for the applicability level,
                                "Threshold_Impact": "Suggested threshold value between 0 and 1",
                            "threshold_reason": "Reason why this threshold value is appropriate"
                              }},
                              {{
                               / "metric_name": "Another metric name",
                               / "metric_description": "Another metric description based on the answer",
                                "validation_steps": "Validation steps for the second metric",
                               / "justification_reason": "Reason for the second metric.",
    							 "category": "Model Quality" or "Trustworthy",
                            "category_reason": "Explanation of why this metric belongs to the selected category",
                            "applicability": "High/Medium/Low",
                                "applicability_reason": "Reason for the applicability level,
                                "Threshold_Impact": "Suggested threshold value between 0 and 1",
                            "threshold_reason": "Reason why this threshold value is appropriate"

                              }}
                            ]
                          }},
                          {{
                            "question_no": "question number from the input",
                            "question": "Another question mentioned",
                            "answer": "Another answer mentioned",
                            "article_laws": [
                              {{
                                "law": "Article name",
                                "article": "Article number",
                                "title": "Title of the Article",
                                "description": "Article description",
                                "reason": "Reason why the article is being identified and mapped for the answers, fetch from input",
                                  "applicability": "High/Medium/Low",
                                "applicability_reason": "Reason for the applicability level
                              }},
                              {{
                                "law": "Second Article name",
                                "article": "Article number",
                                "title": "Title of the Article",
                                "description": "Article description",
                                "reason": "Reason why the article is being identified and mapped for the answers, fetch from input",
                                  "applicability": "High/Medium/Low",
                                "applicability_reason": "Reason for the applicability level
                              }}
                            ],
                            "metrices": [
                              {{
                                "metric_name": "Another metric name",
                                "metric_description": "Another metric description based on the answer",
                                "validation_steps": "Validation steps for this metric",
                                "justification_reason": "Reason for the metric.",
    							 "category": "Model Quality" or "Trustworthy",

                            "category_reason": "Explanation of why this metric belongs to the selected category",
                            "applicability": "High/Medium/Low",
                                "applicability_reason": "Reason for the applicability level,
                                "Threshold_Impact": "Suggested threshold value between 0 and 1",
                            "threshold_reason": "Reason why this threshold value is appropriate"

                              }}
                            ]
                          }}
                        ]
    """

    chat_model = initialize_chat_model()
    response = chat_model.call_as_llm(prompt)

    return response

# Function to load JSON data from a file
def load_json_data(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Main execution
def main():
    # json file path
    json_file_path = '/content/data1.json'

    # Load JSON data
    json_data = load_json_data(json_file_path)

    # Process the input data with the first prompt
    initial_response = process_input_data(json_data)
    print("Initial Response:")
    pprint(initial_response)

    # Process the output from the first response to get the final advisory
    final_response = get_final_advisory(initial_response, json_data)
    print("\nFinal Response:")
    pprint(final_response)

# Run the script
if __name__ == "__main__":
    main()


<ipython-input-2-9f17422a608b>:13: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  return AzureChatOpenAI(
<ipython-input-2-9f17422a608b>:103: LangChainDeprecationWarning: The method `BaseChatModel.call_as_llm` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat_model.call_as_llm(prompt)


Initial Response:
("### Question: Can you briefly describe your application's functionality?\n"
 '#### Metrics:\n'
 '1. **Task Completion Rate (TCR)**  \n'
 '   **Description:** Measures the percentage of calls successfully '
 'transcribed and summarized for agents and customers.  \n'
 '   **Justification Reason:** Ensures the system effectively completes its '
 'primary function of transcribing calls and generating summaries, which is '
 'critical for user satisfaction and operational efficiency.\n'
 '\n'
 '2. **Data Completeness Ratio (DCR)**  \n'
 '   **Description:** Ensures all essential details (e.g., caller information, '
 'travel details) are captured during transcription and summarization.  \n'
 '   **Justification Reason:** Prevents missing information that could lead to '
 'incomplete summaries, ensuring the system provides accurate and actionable '
 'outputs.\n'
 '\n'
 '3. **Correctness/Factuality Rate**  \n'
 '   **Description:** Verifies the accuracy of transcribed data a

In [ ]:
# 1. Based on the justifying reason provided, outline the validation steps for testing the system with the metric you suggested. Ensure that you suggest only the most applicable metric, so the reason and validation steps you provide must be accurate.
#  2. Suggest a threshold for each metric within the range of 0 to 1. Provide a reason for why this threshold is appropriate based on the inputs and the specific context of the question-answer pair. Consider the implications of the metric values, where 0 might indicate poor performance and 1 indicates higher performance, and vice versa based on the metric type (e.g., factuality vs. stereotype).
# 3. Explain the impact on the system if the metrics fall below or exceed the suggested threshold, providing a thorough understanding of the use case.
# 4.** Add a **`category`** field for each metric:
# - **Model Quality:** Metrics that evaluate the quality, performance, or functionality of the application under test.
# - **Trustworthy:** Metrics that assess the ethical behavior, fairness, transparency, and compliance with laws or guidelines.
# 5. **Category Reason (`category_reason`):** Provide the reason why the metric was tagged as either Model Quality or Trustworthy.
# 6. ** Add an **`applicability`** field for each metric:
# - **High:** This metric is highly critical, and it is mandatory for the user, tester, or business analyst to monitor it without fail, as skipping it could significantly impact performance, user experience, or business decisions.
# - **Medium:** This metric is important but not critical. It provides valuable insights that can enhance decision-making, but its absence won’t significantly impact the core functionality or business outcomes.
# - **Low:** This metric is of minimal importance and is primarily for informational or supplementary purposes. It has little to no direct impact on critical functions or business decisions.
# 7. **Applicability Reason (`applicability_reason`):** Provide the reason why the metric has been assigned the specified applicability level, based on its importance in the context of the system and its potential impact on performance, reliability, or compliance.
# Reason for the applicability level,must be inclined towards usecase.
# 8. **Make sure the output looks through all the possible_metrics and  ensure to add all of them to the output.**
# 9. **Make sure the output for all the questions and answers should be present as from the {data}.**






